# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
click = set(df[(df.action == 'click')]['id'].unique())
view = set(df[(df.action == 'view')]['id'].unique())

In [5]:
num_view = len(view)
num_click = len(click)
view_only = len(view - click)
click_only = len(click - view)

print(f'Number of viewers: {num_view}')
print(f'Number of clicks: {num_click}')
print(f'Number of viewers who did not click: {view_only}')
print(f'Number of clicks who did not view: {click_only}')

Number of viewers: 6328
Number of clicks: 1860
Number of viewers who did not click: 4468
Number of clicks who did not view: 0


In [6]:
exp = set(df[(df.group == 'experiment')]['id'].unique())
con = set(df[(df.group == 'control')]['id'].unique())

In [7]:
expcon = len(exp&con)
print(f'Number of overlap between experiment and control ids: {expcon}')

Number of overlap between experiment and control ids: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [8]:
#Your code here
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [9]:
control = df.groupby('group')['action'].value_counts()['control']['view']
control_click = df.groupby('group')['action'].value_counts()['control']['click']
exp = df.groupby('group')['action'].value_counts()['experiment']['view']
exp_click = df.groupby('group')['action'].value_counts()['experiment']['click']

In [10]:
print(f'Control: {control} samples \tExperiment: {exp} samples')
print(f'Control clicks: {control_click} \tExperiment clicks {exp_click}')
print(f'Control click rate: {control_click/control} \tExperiment click rate: {exp_click/exp}')

Control: 3332 samples 	Experiment: 2996 samples
Control clicks: 932 	Experiment clicks 928
Control click rate: 0.2797118847539016 	Experiment click rate: 0.3097463284379172


In [ ]:
'''
Null - experimental click rate <= control click rate
Alternative - experimental click rate > control click rate
alpha = 0.05
'''

In [11]:
df.action.replace({'view':0, 'click':1}, inplace = True)
df = df.sort_values('action', ascending = False)
df.drop_duplicates(subset = ['id'], keep = 'first', inplace = True)
condf = df[df.group == 'control']
expdf = df[df.group == 'experiment']

In [14]:
stats.ttest_ind(condf.action, expdf.action, equal_var = False)[1]/2

0.004466402814337102

In [ ]:
# p-value well below alpha, null hypothesis can be rejected

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [17]:
#Your code here
control_click_rate = control_click/control
expected_clicks = control_click_rate * expdf.shape[0]
expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [18]:
#Your code here
n = expdf.shape[0]
p = control_click_rate
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

In [20]:
z = (exp_click - expected_clicks) / std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [21]:
#Your code here
p = stats.norm.sf(z)
p

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
# Yes p-value is still lower than the alpha, we can reject null hypothesis here. Experimental page seems to bring in more clicks

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.